In [3]:
import pennylane as qml 
import pennylane.numpy as np 

num_wires = 3
dev = qml.device('default.qubit', wires=num_wires)

coeffs = [3, -1]
obs = [qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(0) @ qml.PauliZ(2)]
H = qml.Hamiltonian(coeffs, obs)

@qml.qnode(dev)
def circuit(gamma, alpha, exp=False):
    for i in range(num_wires):
        qml.Hadamard(i)

    for _ in range(3):
        qml.CNOT(wires=[0,1])
        qml.RZ(2*gamma, 1)
        qml.CNOT(wires=[0,1])

        qml.CNOT(wires=[0,2])
        qml.RZ(2*gamma, 2)
        qml.CNOT(wires=[0,2])    

        for i in range(num_wires):
            qml.RX(2*alpha, wires=i)

    if exp: 
        return qml.probs()

    return qml.expval(H)

def cost_function(params):
    gamma, alpha = params[0], params[1]
    return circuit(gamma, alpha)

opt = qml.AdamOptimizer(stepsize=0.01)
params = np.array([0.1, 0.2], requires_grad=True)

for step in range(20):
    params, prev_cost = opt.step_and_cost(cost_function, params)
    print (step, prev_cost, params)

print(params)

probs = circuit(params[0], params[1], True)
for k,v in enumerate(probs):
    print (format(k, '#005b'), v)

0 0.8754299193299608 [0.09 0.21]
1 0.7782623984030669 [0.07999406 0.22000533]
2 0.6763756747471665 [0.06998222 0.23003145]
3 0.5725516650187013 [0.0599684 0.240081 ]
4 0.4695980898674646 [0.04996101 0.25013851]
5 0.37027248001722735 [0.03997351 0.26016503]
6 0.2771639791667506 [0.03002477 0.27009069]
7 0.19253618566684294 [0.02013901 0.27980527]
8 0.1181378133808218 [0.01034512 0.28914792]
9 0.05499700635038507 [0.00067514 0.29790028]
10 0.003232389966376231 [-0.00883841  0.30579012]
11 -0.03806255159496605 [-0.01816608  0.31251375]
12 -0.07079048791417328 [-0.02728552  0.31777934]
13 -0.09762991326905746 [-0.03618503  0.32135951]
14 -0.12165699275383413 [-0.04486589  0.32313149]
15 -0.14593264484162857 [-0.05334286  0.32308772]
16 -0.17319038618708837 [-0.06164291  0.32131798]
17 -0.20568044790221496 [-0.06980277  0.31797643]
18 -0.24513528612357963 [-0.07786558  0.31324823]
19 -0.29278935897310476 [-0.08587737  0.3073236 ]
[-0.08587737  0.3073236 ]
0b000 0.09707951001965114
0b001 0.1